# Building a database for crime reports

In this guided project, we are going to build a database, **crimes_db**,  for storing data related with crimes that occurred in Boston. In this database, we will have a table, **boton_crimes** with appropriate datatypes for storing the data from the *boston.csv* file inside a schema, **crimes**. Besides, we will have 2 groups, **readonly** and **readwrite** with different privileges.

## Create database - crimes_db

In [3]:
import psycopg2
import csv

In [4]:
conn = psycopg2.connect(dbname='dq',
                        user='dq')
conn.autocommit = True

cur = conn.cursor()
cur.execute('CREATE DATABASE crimes_db;')

conn.close()

In [5]:
# connect to database to crimes_db
conn = psycopg2.connect(dbname='crimes_db',
                        user='dq')
cur = conn.cursor()

## Create schema - crimes

In [8]:
cur.execute('CREATE SCHEMA crimes')

### check file before input

Since we are going to input the data from `boston.csv`, we need more information about the columns to determine their types in our table.

In [9]:
with open('boston.csv') as f:
    rows = list(csv.reader(f))
    col_headers = rows[0]
    first_row = rows[1]

In [10]:
print(col_headers)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


In [11]:
def get_col_set(file, col):
    '''    
    get a set of distinct values from the target column
    
    Args:
        file (str): file name of the target data
        col (num): columns index of the target column
        
    Returns:
        set: set of value in the target column
    '''
    with open(file) as f:
        reader = list(csv.reader(f))
        rows = reader[1:]
        
        value = set()
        for row in rows:
            value.add(row[col])
            
    return value

In [12]:
print('Number of distinct values:\n')
for col in range(len(col_headers)):
    length = len(get_col_set('boston.csv', col))
    print('{}: {}'.format(col_headers[col], length))

Number of distinct values:

incident_number: 298329
offense_code: 219
description: 239
date: 1177
day_of_the_week: 7
lat: 18177
long: 18177


We have 7 columns in total. We can basically classify some of their types. For example, we will use **int4** for `incident_number` and `offense_code`, **float8** for `lat` and `long` since they have more than 6 decimal digits, and **date** for `date`. 

However, we still need to decide the length of `description` in order to decide the best type to optimize the storing space. 

####  Max length of the value in ` description`

In [13]:
# column index of 'description'
col_headers.index('description')

2

In [14]:
# get the set of values in description 
desc_set = get_col_set('boston.csv', 2)

In [15]:
# compute the maximum length of any value in description
max_len = 0
max_desc = ''

for ele in desc_set:
    if max_len < len(ele): max_desc = ele
    max_len = max(len(ele), max_len)

print('The max length in description is {}: \n{}'.format(max_len, max_desc))

The max length in description is 58: 
RECOVERED - MV RECOVERED IN BOSTON (STOLEN OUTSIDE BOSTON)


We will use **varchar(100)** for description to ensure there is enough space for input.

#### Enumerated datatype for `day_of_the_week`

`day_of_the_week` has obviously only 7 different values from Sunday to Saturday. A better option is to create an enumerated datatype for this column.

In [16]:
# get values of 'day_of_the_week'
get_col_set('boston.csv', col_headers.index('day_of_the_week'))

{'Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday'}

In [17]:
# create an enumerated datatype for 'day_of_the_week'
cur.execute('''
    CREATE TYPE weekday_enum AS ENUM (
    'Friday', 'Monday', 'Saturday', 'Sunday', 
    'Thursday', 'Tuesday', 'Wednesday'
    );
''')

In summary, we will assign the types as below:

- incident_number: int4
- offense_code: int4
- description: varchar(100)
- date: date
- day_of_the_week: weekday_enum
- lat: float8
- long: float8

Meanwhile, we will set `incident_number` as the primary key since it should be unqiue in the table.

In [18]:
cur.execute('''
    CREATE TABLE crimes.boston_crimes(
        incident_number int4 PRIMARY KEY,
        offense_code int4,
        description varchar(100),
        date date,
        day_of_the_week weekday_enum,
        lat float8,
        long float8
    );
'''
)

## Loading data into crimes.boston_crimes

In [19]:
with open('boston.csv') as f:
    cur.copy_expert('''
    COPY crimes.boston_crimes 
    FROM STDIN WITH CSV HEADER;
    ''', f)

## Create groups - readonly & readwrite

We will create two groups: readonly and readwrite and allow specific privileges as follow:

- readonly: SELECT
- readwrite: SELECT, INSERT, DELETE, UPDATE

By following the least privilege principle, the first step in doing so is to make sure that there are no privileges inherited from the public group and on the public schema. Therefore we will revoke all the privileges of the `public` group on the `public` schema and `crime_db` database.

In [20]:
# Revoke all the privileges of the public group on the public schema
cur.execute('REVOKE ALL ON SCHEMA public FROM public;')
# Revoke all privileges of public on the crime_db database
cur.execute('REVOKE ALL ON DATABASE crimes_db FROM public;')

In [21]:
# crete readonly
cur.execute('CREATE GROUP readonly NOLOGIN;')

# Connection privileges on the crimes_db
cur.execute('GRANT CONNECT ON DATABASE crimes_db TO readonly')

# Usage of the crimes schema
cur.execute('GRANT USAGE ON SCHEMA crimes TO readonly')

# Grant group specific privileges
cur.execute('''
    GRANT SELECT 
    ON ALL TABLES IN SCHEMA crimes TO readonly;
    ''')

In [22]:
# crete readwrite
cur.execute('CREATE GROUP readwrite NOLOGIN;')

# Connection privileges on the crimes_db
cur.execute('GRANT CONNECT ON DATABASE crimes_db TO readwrite')

# Usage of the crimes schema
cur.execute('GRANT USAGE ON SCHEMA crimes TO readwrite')

# Grant group specific privileges
cur.execute('''
    GRANT SELECT, INSERT, DELETE, UPDATE
    ON ALL TABLES IN SCHEMA crimes TO readwrite;
    ''')

## Create users in each group

Now we create some new users and assign them into different groups.

In [23]:
# create user and assign group
cur.execute("CREATE USER data_analyst PASSWORD 'secret1';")
cur.execute('GRANT readonly TO data_analyst;')

In [24]:
# create user and assign group
cur.execute("CREATE USER data_scientist PASSWORD 'secret2';")
cur.execute('GRANT readwrite TO data_scientist;')

In [25]:
# commit all above and close connection
conn.commit()
conn.close()

## Testing database

In [33]:
conn = psycopg2.connect(dbname='crimes_db', user='dq')
cur = conn.cursor()

In [27]:
cur.execute("""
    SELECT rolname, rolsuper, rolcreaterole, rolcreatedb, rolcanlogin FROM pg_roles
    WHERE rolname IN ('readonly', 'readwrite', 'data_analyst', 'data_scientist');
""")
for user in cur:
    print(user)

('readonly', False, False, False, False)
('readwrite', False, False, False, False)
('data_analyst', False, False, False, True)
('data_scientist', False, False, False, True)


### User privileges

In [28]:
cur.execute('''
PREPARE show_privileges(text) AS
SELECT grantee, privilege_type
FROM information_schema.table_privileges
WHERE grantee = $1;
''')

In [29]:
group = ['readwrite']

cur.execute('EXECUTE show_privileges(%s);', group)
cur.fetchall()

[('readwrite', 'INSERT'),
 ('readwrite', 'SELECT'),
 ('readwrite', 'UPDATE'),
 ('readwrite', 'DELETE')]

In [30]:
group = ['readonly']

cur.execute('EXECUTE show_privileges(%s);', group)
cur.fetchall()

[('readonly', 'SELECT')]

### Table - crimes.boston_crimes

In [39]:
cur.execute('''
SELECT *
FROM crimes.boston_crimes
LIMIT 3;
''')

cur.fetchall()

[(1,
  619,
  'LARCENY ALL OTHERS',
  datetime.date(2018, 9, 2),
  'Sunday',
  42.35779134,
  -71.13937053),
 (2,
  1402,
  'VANDALISM',
  datetime.date(2018, 8, 21),
  'Tuesday',
  42.30682138,
  -71.06030035),
 (3,
  3410,
  'TOWED MOTOR VEHICLE',
  datetime.date(2018, 9, 3),
  'Monday',
  42.34658879,
  -71.07242943)]

In [13]:
conn.close()